In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

# from sklearn.utils import shuffle
# from sktime.classification.compose import ColumnEnsembleClassifier
# from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
# from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor

import os
import mne
import pandas as pd
from mne.externals.pymatreader import read_mat
from sklearn.utils import shuffle

import datetime

In [2]:
data_path = 'C:\\Users\\frank\\code\\NeuroErgonomics_Hackathon_2021'
#'/home/dcas/l.darmet/data/contest/comeptition_done'
n_subs = 15
diff = ['MATBeasy', 'MATBmed', 'MATBdiff']

ch_slice = ['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'AF3', 'AFz', 'AF4','FP1', 'FP2', 'FPz']
# ch_slice = ['FP1']

n_estimators = 300 
narrow_feature_space = False

data_params =   {'n_estimators': n_estimators,
                    'narrow_feature_space': narrow_feature_space,
                }
    

    

In [3]:
#Train on first session, test on second session
train_accuracy = list()
accuracies = list()
# for sub_n, session_n in itertools.product(range(n_subs), range(n_sessions)):
for sub_n in range(n_subs):
    session_n = 0
    epochs_data = []
    labels = []
    for lab_idx, level in enumerate(diff):
        sub = 'P{0:02d}'.format(sub_n+1)
        sess = f'S{session_n+1}'
        path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
        # Read the epoched data with MNE
        epochs = mne.io.read_epochs_eeglab(path, verbose=False)
        # You could add some pre-processing here with MNE
        # We will just select some channels (mostly frontal ones)
        if (narrow_feature_space):
            epochs = epochs.drop_channels(list(set(epochs.ch_names) -set(ch_slice)))

        # Get the data and concatenante with others MATB levels
        tmp = epochs.get_data()
        epochs_data.extend(tmp)
        labels.extend([lab_idx]*len(tmp))
    
        X = np.array(epochs_data)
    labels = np.array(labels)
    y = labels
    x_names = [f'dim_{x}' for x in range(X.shape[1])]    
    X_df = pd.DataFrame(columns = x_names)

    sample =  0 
    for sample in range(X.shape[0]):
        data = X[sample,:,:]
        list_of_series = []
        for xx in range(X.shape[1]):
            list_of_series.append(X[sample,xx,:])

        X_df = X_df.append(pd.DataFrame([list_of_series], columns = x_names))

    X_df.reset_index(drop = True)


    session_n = 1
    epochs_data = []
    labels = []
    for lab_idx, level in enumerate(diff):
        sub = 'P{0:02d}'.format(sub_n+1)
        sess = f'S{session_n+1}'
        path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
        # Read the epoched data with MNE
        epochs = mne.io.read_epochs_eeglab(path, verbose=False)
        # You could add some pre-processing here with MNE
        # We will just select some channels (mostly frontal ones)
        if (narrow_feature_space):
            epochs = epochs.drop_channels(list(set(epochs.ch_names) -set(ch_slice)))

        # Get the data and concatenante with others MATB levels
        tmp = epochs.get_data()
        epochs_data.extend(tmp)
        labels.extend([lab_idx]*len(tmp))
    
        X_s2 = np.array(epochs_data)
    labels = np.array(labels)
    y_s2 = labels


    Xs2_df = pd.DataFrame(columns = x_names)

    sample =  0 
    for sample in range(X_s2.shape[0]):
        data = X_s2[sample,:,:]
        list_of_series = []
        for xx in range(X_s2.shape[1]):
            list_of_series.append(X_s2[sample,xx,:])

        Xs2_df = Xs2_df.append(pd.DataFrame([list_of_series], columns = x_names))

    Xs2_df.reset_index(drop = True)
        
    X_train, X_test, y_train, y_test = X_df, Xs2_df, y, y_s2
    
    X_test, y_test = shuffle(X_test, y_test, random_state=0)
    
#This model gave the best perfromance over the full group
    steps = [
        ("concatenate", ColumnConcatenator()),
        ("classify", TimeSeriesForestClassifier(n_estimators=data_params['n_estimators'], random_state=0, n_jobs=-1)),
        ]
    clf = Pipeline(steps)
#This model gave the best performance over 4 participants- never ran to completion
#     clf = make_pipeline(
#         ColumnConcatenator(), TSFreshFeatureExtractor(n_jobs=-1, show_warnings=False), RandomForestClassifier(random_state=0)
#         )

    ###____

    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    accuracies.append(acc)
    print(acc)
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") )
    
print(f'the average accuracy is {np.mean(accuracies)}')

0.4317673378076063
2021-07-25 15:21:12
0.5279642058165548
2021-07-25 15:51:12
0.5480984340044742
2021-07-25 16:21:10
0.42058165548098436
2021-07-25 16:51:15
0.4161073825503356
2021-07-25 17:21:17
0.5570469798657718
2021-07-25 17:51:21
0.5525727069351231
2021-07-25 18:21:47
0.5391498881431768
2021-07-25 18:51:54
0.5883668903803132
2021-07-25 19:23:05
0.5055928411633109
2021-07-25 19:52:59
0.5816554809843401
2021-07-25 20:22:52
0.49217002237136465
2021-07-25 20:52:53
0.41834451901565994
2021-07-25 21:22:46
0.4854586129753915
2021-07-25 21:52:40
0.47651006711409394
2021-07-25 22:23:15
the average accuracy is 0.5027591349739001


In [7]:
# Train on part of the data and test on the other part of the data

#Train on first session, test on second session
train_accuracy = list()
accuracies = list()
# for sub_n, session_n in itertools.product(range(n_subs), range(n_sessions)):
for sub_n in range(n_subs):
    session_n = 0
    epochs_data = []
    labels = []
    for lab_idx, level in enumerate(diff):
        sub = 'P{0:02d}'.format(sub_n+1)
        sess = f'S{session_n+1}'
        path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
        # Read the epoched data with MNE
        epochs = mne.io.read_epochs_eeglab(path, verbose=False)
        # You could add some pre-processing here with MNE
        # We will just select some channels (mostly frontal ones)
        if (narrow_feature_space):
            epochs = epochs.drop_channels(list(set(epochs.ch_names) -set(ch_slice)))

        # Get the data and concatenante with others MATB levels
        tmp = epochs.get_data()
        epochs_data.extend(tmp)
        labels.extend([lab_idx]*len(tmp))
    
        X = np.array(epochs_data)
    labels = np.array(labels)
    y = labels
    x_names = [f'dim_{x}' for x in range(X.shape[1])]    
    X_df = pd.DataFrame(columns = x_names)

    sample =  0 
    for sample in range(X.shape[0]):
        data = X[sample,:,:]
        list_of_series = []
        for xx in range(X.shape[1]):
            list_of_series.append(X[sample,xx,:])

        X_df = X_df.append(pd.DataFrame([list_of_series], columns = x_names))

    X_df.reset_index(drop = True)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_df, y, test_size=0.33, random_state=42)
    
    #This model gave the best perfromance over the full group
    steps = [
        ("concatenate", ColumnConcatenator()),
        ("classify", TimeSeriesForestClassifier(n_estimators=data_params['n_estimators'], random_state=0, n_jobs=-1)),
        ]
    clf = Pipeline(steps)
#This model gave the best performance over 4 participants- never ran to completion
#     clf = make_pipeline(
#         ColumnConcatenator(), TSFreshFeatureExtractor(n_jobs=-1, show_warnings=False), RandomForestClassifier(random_state=0)
#         )

    ###____

    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    accuracies.append(acc)
    print(acc)
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") )
    
print(f'the average accuracy is {np.mean(accuracies)}')

0.7635135135135135
2021-07-25 22:48:06
0.6756756756756757
2021-07-25 23:02:53
0.6621621621621622
2021-07-25 23:17:42
0.7027027027027027
2021-07-25 23:32:29
0.6216216216216216
2021-07-25 23:47:17
0.7432432432432432
2021-07-26 00:02:05
0.7364864864864865
2021-07-26 00:16:52
0.6081081081081081
2021-07-26 00:31:39
0.6351351351351351
2021-07-26 00:46:27
0.6756756756756757
2021-07-26 01:01:15
0.6351351351351351
2021-07-26 01:16:02
0.6148648648648649
2021-07-26 01:30:49
0.6486486486486487
2021-07-26 01:45:38
0.6216216216216216
2021-07-26 02:00:26
0.6013513513513513
2021-07-26 02:15:14
the average accuracy is 0.6630630630630631
